In [351]:
import pandas as pd
from pandas import ExcelWriter
import matplotlib.pyplot as plt

In [352]:
# import the data
sdg_in = pd.read_excel('SDG-DEA_indicators_V3_20200428.xlsx',
                       sheet_name='SDG-DEA FINAL',
                       header=4,
                       index_col=0,
                       usecols='B,T:DQ',
                       skiprows=[5]
                       )

dir_sdg_in = pd.read_excel('SDG-DEA_indicators_V3_20200428.xlsx',
                             sheet_name='SDG-DEA FINAL',
                             usecols='T:DQ',
                             nrows=2,
                             skiprows=[0,1],
                             header=None
                             )

In [353]:
# prepare the data

# make a copy to be able to repeat operations without reloading from excel (we're dropping stuff)
inds = sdg_in.copy()

# cutoff for missing data (i.e. we keep if fewer than [cutoff] are missing)
cutoff = 0.5

# find countries missing more than [cutoff] indicators
miss_cnt_perc = (inds.isna().sum(axis=1)/inds.shape[1])
drop_bool = miss_cnt_perc >= cutoff
drop_cnt = drop_bool[drop_bool].index

# find indicators missing more than [cutoff] countries
miss_ind_perc = (inds.isna().sum(axis=0)/inds.shape[0])
drop_bool = miss_ind_perc >= cutoff
drop_ind = drop_bool[drop_bool].index

# drop countries and indicators missing the cutoff
inds.drop(labels = drop_cnt, inplace=True)
inds.drop(columns = drop_ind, inplace=True)

# fill remaining missing values with indicator means
inds.fillna(inds.mean(axis=0), inplace=True)


# modify direction and sdg data
dir_sdg = dir_sdg_in.copy()
dir_sdg.index = ['direction','sdg']
dir_sdg = dir_sdg.fillna(method='ffill', axis=1)
dir_sdg.columns = sdg_in.columns
dir_sdg.drop(columns = drop_ind, inplace=True)

In [390]:
# export to excel file, 1x all sdgs, 1x for each sdg

sheets = ['all',*dir_sdg.loc['sdg',:].unique()]
dummy = pd.DataFrame([1]*len(inds.index), index=inds.index, columns=['dummy']) # dummy column to use if no inputs or outputs exist for an SDG

with ExcelWriter('sdg_gms_input.xlsx') as writer:
    
    for s in sheets:
        
        # one sheet for all sdgs, then one sheet for each sdg
        if s == 'all':
            cols_in = dir_sdg.columns[dir_sdg.loc['direction',:] == -1]
            cols_out = dir_sdg.columns[dir_sdg.loc['direction',:] == 1]
        else:
            cols_in = dir_sdg.columns[(dir_sdg.loc['direction',:] == -1) & (dir_sdg.loc['sdg'] == s)]
            cols_out = dir_sdg.columns[(dir_sdg.loc['direction',:] == 1) & (dir_sdg.loc['sdg'] == s)]
            
        # if no inputs exist, add dummy input
        if len(cols_in) == 0:
            dummy.to_excel(writer,sheet_name=s, startrow=0)
        else:
            inds[cols_in].to_excel(writer,sheet_name=s, startrow=0)
        
        # if no outputs exist, write dummy output
        if len(cols_out) == 0:
            dummy.to_excel(writer,sheet_name=s, startrow=inds.shape[0]+2)
        else:
            inds[cols_out].to_excel(writer,sheet_name=s, startrow=inds.shape[0]+2)

    writer.save()

Index(['sdg1_wpc', 'sdg1_320pov', 'sdg1_11_13_affected', 'sdg2_undernsh',
       'sdg2_stunting', 'sdg2_wasting', 'sdg2_obesity', 'sdg2_nue',
       'sdg2_trophic', 'sdg3_matmort', 'sdg3_neonat', 'sdg3_u5mort', 'sdg3_tb',
       'sdg3_malaria', 'sdg3_hiv', 'sdg3_ncds', 'sdg3_suicide', 'sdg3_alcohol',
       'sdg3_pollmort', 'sdg3_traffic', 'sdg3_fertility', 'sdg6_freshwat',
       'sdg6_gwd', 'sdg7_enerintensity', 'sdg7_co2twh', 'sdg8_slavery',
       'sdg16_clabor', 'sdg8_unemp', 'sdg8_impacc', 'sdg10_adjgini',
       'sdg10_fsi3', 'sdg10_fsi4', 'sdg10_fsi7', 'sdg10_refugees',
       'sdg11_pm25', 'sdg1_11_13_affected.1', 'sdg12_msw', 'sdg12_ewaste',
       'sdg12_prodso2', 'sdg12_so2', 'sdg12_prodnitro', 'sdg12_reacnitro',
       'sdg13_co2pc', 'sdg13_co2import', 'sdg13_affected',
       'sdg1_11_13_affected.2', 'sdg13_co2export', 'sdg14_fishstocks',
       'sdg14_trawl', 'sdg15_forchg', 'sdg15_impbiothr', 'sdg16_homicides',
       'sdg16_detain', 'sdg16_weaponsexp', 'sdg16_rsf', 'sd